In [ ]:
! git clone https://github.com/spMohanty/PlantVillage-Dataset

Cloning into 'PlantVillage-Dataset'...
remote: Enumerating objects: 163235, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.


In [ ]:
DATASET_ROOT = "/content/PlantVillage-Dataset/raw/color"

Apple -{Apple-scab , Black_rot, Cedar_apple_rust, healthy}

Blueberry—healthy

Cherry -{powdery_mildew, healthy}
Corn- {cercospora_leaf_spot Gray_leaf_spot, Common-rust—, Northern_Leaf_Blight, healthy}

Grape -{Black-rot, Esca_(Black_Measles), Leaf-blight_(lsariopsis_Leaf_spot), healthy}
Orange -{Haunglongbing_(Citrus-greening)}

Peach -{Bacterial_spot, healthy}

Pepper -{bell_Bacterial_spot}

Potato -{Farly_blight, healthy}

Raspberry -{healthy}

Soybean -{healthy}
squash -{powdery-mildew}
Strawberry {Leaf_scorch, healthy}
Tomato_Bacterial_spot
Tomato—Late_blight
Tomato—Leaf-Mold
_septoria_leaf_spot
Tomato
Tomato_Spider_mites Two-spotted_spider_mite
Tomato_Target_Spot
Tomato_Tomato_Yellow_Leaf_CurLVirus
Tomato_mosaic_virus
Tomato
Tomato—healthy

In [ ]:
!ls $DATASET_ROOT | head

In [ ]:
# !pip install tensorflow tensorflow-datasets matplotlib seaborn scikit-learn

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
img_size = (224, 224)
batch_size = 16

datagen = ImageDataGenerator(
    validation_split=0.2,
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)

In [ ]:
train_gen = datagen.flow_from_directory(
    DATASET_ROOT,
    target_size=img_size,
    batch_size=batch_size,
    subset="training",
    class_mode="categorical"
)

In [ ]:
val_gen = datagen.flow_from_directory(
    DATASET_ROOT,
    target_size=img_size,
    batch_size=batch_size,
    subset="validation",
    class_mode="categorical"
)

In [ ]:
num_classes = len(train_gen.class_indices)
print("Classes:", train_gen.class_indices)

Build Model (Transfer Learning)

In [ ]:
base_model = tf.keras.applications.EfficientNetB0(
    input_shape=(224,224,3),
    include_top=False,
    weights="imagenet"
)
base_model.trainable = False  # Freeze base initially


Building custom classifier head

In [ ]:
x = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
# Batch Normalisation
x = tf.keras.layers.BatchNormalization()(x)

In [ ]:
# Dropout for regularization
x = tf.keras.layers.Dropout(0.4)(x)

In [ ]:
x = tf.keras.layers.Dense(256, activation="relu")(x)
# Batch Normalisation after dense layer
x = tf.keras.layers.BatchNormalization()(x)

In [ ]:
x = tf.keras.layers.Dropout(0.4)(x)
outputs = tf.keras.layers.Dense(num_classes, activation="softmax")(x)

In [ ]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(2, 2),
    Dropout(0.2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Dropout(0.2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

Compile Model

In [ ]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [ ]:
import os
import shutil
import random

# Create train/val directories
train_dir = "/content/PlantVillage-Dataset/train"
val_dir = "/content/PlantVillage-Dataset/val"

os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

In [ ]:
# Loop over each class folder
dataset_dir = "/content/PlantVillage-Dataset/raw/color"
for class_name in os.listdir(dataset_dir):
    class_path = os.path.join(dataset_dir, class_name)
    if os.path.isdir(class_path):
        images = os.listdir(class_path)
        random.shuffle(images)
        split = int(len(images) * 0.8)  # 80% train, 20% val

        train_class_dir = os.path.join(train_dir, class_name)
        val_class_dir = os.path.join(val_dir, class_name)
        os.makedirs(train_class_dir, exist_ok=True)
        os.makedirs(val_class_dir, exist_ok=True)

        # Move images
        for img in images[:split]:
            shutil.copy(os.path.join(class_path, img), train_class_dir)
        for img in images[split:]:
            shutil.copy(os.path.join(class_path, img), val_class_dir)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    '/content/PlantVillage-Dataset/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)
val_generator = test_datagen.flow_from_directory(
    '/content/PlantVillage-Dataset/val',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=25
)

In [ ]:
model.summary()

In [ ]:
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()